<a href="https://colab.research.google.com/github/DeveloperVivek9/Calorie-Tracker/blob/main/Deployment/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install import-ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import copy
import cv2
from google.colab import drive
import itertools
import import_ipynb
import numpy as np
import pandas as pd
import os
import re
import shutil
import time

In [ ]:
#Generate classes list
#Preprocessing
#Augmentation
#If n(batch==None) == threshold -> train/test split -> train
#save latest model as x.pth
#train on that

In [3]:
import Data_Augmentation
import Create_csv
import Data_Preprocessing
import Training

importing Jupyter notebook from Data_Augmentation.ipynb
albumentations==1.3.0 is successfully installed
importing Jupyter notebook from Create_csv.ipynb
importing Jupyter notebook from Data_Preprocessing.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
importing Jupyter notebook from Training.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
#ONLY FOR TESTING PURPOSES !!!!!!!! REMOVES VALUABLE FILES TO RECREATE PROCESSES
def main():
  drive.mount('/content/drive')
  base_path="/content/drive/MyDrive/Experiment/"
  dataset_path=base_path+'Unannotated/'
  disparity_dir=base_path+'Disparity/'
  destination_dir=base_path+'Annotated/'
  for i in [dataset_path,disparity_dir,destination_dir]:
    try:
      shutil.rmtree(i)
    except:
      continue
  for i in ['Image Metadata.csv','Bad Files.csv','Classes List.csv']:
    try:
      os.remove(base_path+i)
    except:
      continue
  shutil.copytree(base_path+'Backup',dataset_path)
  image_extension_list=['.jpg', '.png', '.jpeg', '.JPG', '.webp', '.cms', '.jfif']
  #Creates classes list csv
  Create_csv.generate_classes_list(base_path,"Franchise Food Items..xlsx")
  while(True):
    #Adds annotated data onto csv file. Creates Image Metadata csv
    Data_Preprocessing.data_entry(dataset_path,destination_dir,disparity_dir,image_extension_list,base_path+'Image Metadata.csv',base_path+'Bad Files.csv',base_path+"Classes_List.csv")
    #Augments annotated images.
    Data_Augmentation.Augmentation_function(base_path,'Image Metadata.csv')
    #Gets count of images where batch == None
    metadata_df=pd.read_csv(base_path+'Image Metadata.csv')
    index=metadata_df[metadata_df["Batch"].isnull()].index
    print(index)
    if(len(index)>=1):
      Create_csv.generate_train_test_csv(base_path,'Image Metadata.csv')
      Training.Initial_Training(base_path)
      metadata_df.loc[index,"Batch"]=0
      metadata_df.to_csv(base_path+'Image Metadata.csv',index=False)
    index=metadata_df[metadata_df["Batch"].isnull()].index
    if(len(index)>=1):
      Create_csv.generate_train_test_csv(base_path,'Image Metadata.csv')
      count=1
      if(len(metadata_df.loc[index,"Image ID"].unique())>=1000): 
        #Splits images not having been trained into train and test
        Create_csv.generate_train_test_csv(base_path,'Image Metadata.csv')
        Training.Online_Training(base_path,count)
        metadata_df.loc[index,"Batch"]=count
        metadata_df.to_csv(base_path+'Image Metadata.csv',index=False)
        count+=1
    del(metadata_df)
    # creating a time delay of 1 min
    print("LOOP DONE. SLEEPING.")
    time.sleep(60)

In [23]:
if __name__=="__main__":
    main()